In [38]:
"""IMPORT BIBLIOTEK"""
import mne 
import matplotlib.pyplot as plt
import numpy as np 
from mne.preprocessing import ICA, create_eog_epochs, create_ecg_epochs #funkcje ica
from mne_icalabel import label_components #ica-label algorytm
import onnxruntime #biblioteka do obsługi ica-label
#backend matplotliba 
%matplotlib qt 
"""WSTĘPNE USTAWIENIA PRZED WCZYTANIEM SYGNAŁU"""
d_path = "eeg_files/20241017_kp.bdf" #ścieżka do pliku
ch_eog = ["EXG1","EXG2"] #zdefiniowanie dedykowanych kanałów EOG
ch_ecg = ["EXG3","EXG4"] #zdefiniowanie dedykowanych kanałów ECG
ch_exclude = [f"EXG{i}" for i in range(5,9)] #wykluczenie kanałów niewykorzystanych w badaniu
"""ZAŁADOWANIE SYGNAŁU I USTAWIENIE UKŁADU ELEKTROD"""
raw = mne.io.read_raw_bdf(
    d_path, preload = True, eog = ch_eog, misc = ch_ecg, exclude = ch_exclude 
)
raw.set_montage("biosemi64") #ustawienie montazu biosemi


Extracting EDF parameters from /Users/szymbierz/Desktop/wszystko/notebooks/statystyka/inne/eeg/eeg_files/20241017_kp.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 2928639  =      0.000 ...  1430.000 secs...


<RawEDF | 20241017_kp.bdf, 69 x 2928640 (1430.0 s), ~1.51 GiB, data loaded>

In [39]:
notch = 50 
bandpass_low = 0.1 
bandpass_high = 40
raw.filter(bandpass_low,bandpass_high ) 
raw.notch_filter(notch)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 67585 samples (33.000 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.9s


Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 49.38
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 49.12 Hz)
- Upper passband edge: 50.62 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 50.88 Hz)
- Filter length: 13517 samples (6.600 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.7s


<RawEDF | 20241017_kp.bdf, 69 x 2928640 (1430.0 s), ~1.51 GiB, data loaded>

In [41]:
"""OBLCZENIE MACIERZY KORELACJI WSZYSTKICH KANAŁÓW"""
data = raw.copy().pick("eeg").get_data()
corr = np.corrcoef(data)
print(corr)

[[1.         0.8621947  0.88410815 ... 0.3044071  0.40796165 0.47306334]
 [0.8621947  1.         0.84240092 ... 0.25766983 0.34838442 0.39526421]
 [0.88410815 0.84240092 1.         ... 0.1726086  0.2508038  0.27991012]
 ...
 [0.3044071  0.25766983 0.1726086  ... 1.         0.87948592 0.89195904]
 [0.40796165 0.34838442 0.2508038  ... 0.87948592 1.         0.87228335]
 [0.47306334 0.39526421 0.27991012 ... 0.89195904 0.87228335 1.        ]]


In [42]:
"""EKSTRAKCJA NAZW OSI WYKRESU"""
eeg_channels = raw.ch_names
channels_to_exclude = ['EXG1', 'EXG2', 'EXG3', 'EXG4','Status']
for channel_to_remove in channels_to_exclude:
    if channel_to_remove in eeg_channels:
        eeg_channels.remove(channel_to_remove) 


In [30]:
pip install seaborn 

  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
Using cached seaborn-0.13.2-py3-none-any.whl (294 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 2.5 MB/s eta 0:00:000:00:010:00:0102

[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.4.1_1/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
"""HEATMAPA KORELACJI WSZYSTKICH KANAŁÓW"""

plt.figure(figsize=(10, 10))
plt.imshow(corr_2, cmap='RdBu_r',interpolation="nearest")
plt.colorbar(label='Pearson r')
plt.xticks(range(len(eeg_channels)), eeg_channels, rotation=90)
plt.yticks(range(len(eeg_channels)), eeg_channels)
plt.tight_layout()
plt.show()

In [59]:
"""ŚREDNIA KORELACJA WSZYSTKICH KANAŁÓW"""
mean_corr = corr.mean(axis=1)
sorted_idx = np.argsort(mean_corr)
for idx in sorted_idx:
    print(f"{eeg_channels[idx]}: średnie r = {mean_corr[idx]:.2f}")

POz: średnie r = -0.34
Oz: średnie r = -0.23
PO3: średnie r = -0.00
O1: średnie r = 0.01
C3: średnie r = 0.01
AFz: średnie r = 0.02
CP1: średnie r = 0.25
T7: średnie r = 0.36
P9: średnie r = 0.38
PO7: średnie r = 0.38
AF3: średnie r = 0.39
Iz: średnie r = 0.40
F8: średnie r = 0.41
FT8: średnie r = 0.42
FT7: średnie r = 0.42
P10: średnie r = 0.42
F7: średnie r = 0.43
P1: średnie r = 0.44
C1: średnie r = 0.44
Fpz: średnie r = 0.44
Fp2: średnie r = 0.44
Fp1: średnie r = 0.45
Cz: średnie r = 0.46
AF8: średnie r = 0.46
AF7: średnie r = 0.47
TP8: średnie r = 0.47
CP3: średnie r = 0.47
P8: średnie r = 0.48
CP5: średnie r = 0.48
TP7: średnie r = 0.48
C5: średnie r = 0.48
Fz: średnie r = 0.49
P3: średnie r = 0.50
AF4: średnie r = 0.50
PO8: średnie r = 0.50
F4: średnie r = 0.50
P7: średnie r = 0.50
F6: średnie r = 0.51
FC5: średnie r = 0.51
FC1: średnie r = 0.51
P2: średnie r = 0.52
PO4: średnie r = 0.52
C2: średnie r = 0.52
O2: średnie r = 0.52
CP2: średnie r = 0.52
F5: średnie r = 0.52
F2: śre

In [70]:
import pandas as pd
mean_corr_rounded = mean_corr.round(2)
mean_corr_df = pd.DataFrame(mean_corr_rounded[:18],index=eeg_channels[:18],columns=["średnie r"])
mean_corr_df

,średnie r
Fp1,0.45
AF7,0.47
AF3,0.39
F1,0.54
F3,0.54
F5,0.52
F7,0.43
FT7,0.42
FC5,0.51
FC3,0.55


In [80]:
mean_corr_df_2 = pd.DataFrame(mean_corr_rounded[18:37],index=eeg_channels[18:37],columns=["średnie r"])
mean_corr_df_2

,średnie r
CP1,0.25
P1,0.44
P3,0.50
P5,0.53
P7,0.50
P9,0.38
PO7,0.38
PO3,-0.00
O1,0.01
Iz,0.40


In [84]:
mean_corr_df_3 = pd.DataFrame(mean_corr_rounded[37:55],index=eeg_channels[37:55],columns=["średnie r"])
mean_corr_df_3

,średnie r
Fz,0.49
F2,0.53
F4,0.50
F6,0.51
F8,0.41
FT8,0.42
FC6,0.53
FC4,0.54
FC2,0.55
FCz,0.58


In [87]:
mean_corr_df_4 = pd.DataFrame(mean_corr_rounded[55:],index=eeg_channels[55:],columns=["średnie r"])
mean_corr_df_4

,średnie r
CP2,0.52
P2,0.52
P4,0.56
P6,0.54
P8,0.48
P10,0.42
PO8,0.50
PO4,0.52
O2,0.52


In [89]:
from pyprep.prep_pipeline import PrepPipeline
"""DO PYPREP""" 
raw_for_pyprep=raw.copy() #utworzenie kopi do pyprep
sample_rate = raw_for_pyprep.info["sfreq"] #pobranie informacji o częstotliwości próbkowania
montage = mne.channels.make_standard_montage("biosemi64")
prep_params = {
    "ref_chs": "eeg",
    "reref_chs": "eeg",
    "line_freqs": np.arange(50, sample_rate / 2, 50),
}
prep = PrepPipeline(raw_for_pyprep, prep_params, montage)
prep.fit()

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


RuntimeError: info channel name inconsistency detected, please notify MNE-Python developers

In [ ]:
print("Złe kanały: {}".format(prep.interpolated_channels))
print("Oryginalne złe kanały: {}".format(prep.noisy_channels_original["bad_all"]))
print("Złe kanały po interpolacji: {}".format(prep.still_noisy_channels))

In [90]:
\

['Fp1', 'AF7', 'AF3', 'F1', 'F3', 'F5', 'F7', 'FT7', 'FC5', 'FC3', 'FC1', 'C1', 'C3', 'C5', 'T7', 'TP7', 'CP5', 'CP3', 'CP1', 'P1', 'P3', 'P5', 'P7', 'P9', 'PO7', 'PO3', 'O1', 'Iz', 'Oz', 'POz', 'Pz', 'CPz', 'Fpz', 'Fp2', 'AF8', 'AF4', 'AFz', 'Fz', 'F2', 'F4', 'F6', 'F8', 'FT8', 'FC6', 'FC4', 'FC2', 'FCz', 'Cz', 'C2', 'C4', 'C6', 'T8', 'TP8', 'CP6', 'CP4', 'CP2', 'P2', 'P4', 'P6', 'P8', 'P10', 'PO8', 'PO4', 'O2']
